In [3]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_zseZCNwp4IA1s9Dc8EuA8Lyc', name='get_weather', type='function_call', id='fc_68040508d8288192b930c012d459fd6f0cc15c2f5b761137', status='completed')]


# Linh tinh test


In [4]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, formats output and counts words.
    Output: Dictionary containing:
            - status ('success' or 'error')
            - transcript text and word count on success
            - error message on failure
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            word_count = len(transcript_text.split())
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {
                "status": "success", 
                "transcript": transcript_text,
                "word_count": word_count
            }
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

import logging
logger = logging.getLogger(__name__)
result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


{'status': 'success',
 'transcript': 'cool okay let\'s get on with it so hi everyone my name is Elan i\'m on the developer experience team at OpenAI um unfortunately I can\'t be there in person as much as I would love to i\'m in a wedding in Costa Rica um which is happening later today so I just wanted to take this opportunity to just talk through um one of my favorite concepts in maybe all of like AI and language models so uh title of this talk is function calling is all you need it\'s a talk workshop there\'s going to be um a lot of coding please save your questions no I\'m kidding like just interrupt at any point uh we have a Slack send them here um yeah just send them at any point uh and if you want to like unmute yourself and or raise your hand I\'ll call you off like as we go the idea is to keep this super super dynamic um since we have a bit of time um I\'ll be fielding a lot of like questions and requests and trying to be coding as much as possible so yeah this talk is going to

In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, formats output and counts words.
    Output: Dictionary containing:
            - status ('success' or 'error')
            - transcript text and word count on success
            - error message on failure
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            word_count = len(transcript_text.split())
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {
                "status": "success", 
                "transcript": transcript_text,
                "word_count": word_count
            }
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 
    
import logging
logger = logging.getLogger(__name__)
result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


{'status': 'success',
 'transcript': 'Cool. Okay, let\'s get on with it. So, hi everyone. My name is Elan. I\'m on the developer experience team at OpenAI. Um, unfortunately, I can\'t be there in person as much as I would love to. I\'m in a wedding in Costa Rica um which is happening later today. So, I just wanted to take this opportunity to just talk through um one of my favorite concepts in maybe all of like AI and language models. So, uh title of this talk is function calling is all you need. It\'s a talk workshop. There\'s going to be um a lot of coding. Please save your questions. No, I\'m kidding. Like just interrupt at any point. Uh we have a Slack. Send them here. Um yeah, just send them at any point. Uh and if you want to like unmute yourself and or raise your hand, I\'ll call you off like as we go. The idea is to keep this super super dynamic. Um since we have a bit of time, um I\'ll be fielding a lot of like questions and requests and trying to be coding as much as possible.

In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, and formats the output.
    Output: Dictionary containing status ('success' or 'error') and either
            the 'transcript' or an error 'message'.
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {"status": "success", "transcript": transcript_text}
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

import logging
logger = logging.getLogger(__name__)
result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


{'status': 'success',
 'transcript': 'Cool. Okay, let\'s get on with it. So, hi everyone. My name is Elan. I\'m on the developer experience team at OpenAI. Um, unfortunately, I can\'t be there in person as much as I would love to. I\'m in a wedding in Costa Rica um which is happening later today. So, I just wanted to take this opportunity to just talk through um one of my favorite concepts in maybe all of like AI and language models. So, uh title of this talk is function calling is all you need. It\'s a talk workshop. There\'s going to be um a lot of coding. Please save your questions. No, I\'m kidding. Like just interrupt at any point. Uh we have a Slack. Send them here. Um yeah, just send them at any point. Uh and if you want to like unmute yourself and or raise your hand, I\'ll call you off like as we go. The idea is to keep this super super dynamic. Um since we have a bit of time, um I\'ll be fielding a lot of like questions and requests and trying to be coding as much as possible.

In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, and formats the output.
    Output: Dictionary containing status ('success' or 'error') and either
            the 'transcript' or an error 'message'.
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {"status": "success", "transcript": transcript_text}
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

import logging
logger = logging.getLogger(__name__)
result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


{'status': 'success',
 'transcript': 'Cool. Okay, let\'s get on with it. So, hi everyone. My name is Elan. I\'m on the developer experience team at OpenAI. Um, unfortunately, I can\'t be there in person as much as I would love to. I\'m in a wedding in Costa Rica um which is happening later today. So, I just wanted to take this opportunity to just talk through um one of my favorite concepts in maybe all of like AI and language models. So, uh title of this talk is function calling is all you need. It\'s a talk workshop. There\'s going to be um a lot of coding. Please save your questions. No, I\'m kidding. Like just interrupt at any point. Uh we have a Slack. Send them here. Um yeah, just send them at any point. Uh and if you want to like unmute yourself and or raise your hand, I\'ll call you off like as we go. The idea is to keep this super super dynamic. Um since we have a bit of time, um I\'ll be fielding a lot of like questions and requests and trying to be coding as much as possible.

In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, and formats the output.
    Output: Dictionary containing status ('success' or 'error') and either
            the 'transcript' or an error 'message'.
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {"status": "success", "transcript": transcript_text}
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, and formats the output.
    Output: Dictionary containing status ('success' or 'error') and either
            the 'transcript' or an error 'message'.
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {"status": "success", "transcript": transcript_text}
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


NameError: name 'logger' is not defined

In [ ]:
def get_youtube_transcript(youtube_url: str):
    """
    Tool function to fetch the transcript of a YouTube video.
    Input: YouTube video URL string.
    Process: Extracts video ID, calls YouTubeTranscriptApi, and formats the output.
    Output: Dictionary containing status ('success' or 'error') and either
            the 'transcript' or an error 'message'.
    """
    import re # Moved import inside
    from youtube_transcript_api import YouTubeTranscriptApi # Moved import inside
    from typing import Dict, Any # Keep for return type annotation

    logger.info(f"Tool 'get_youtube_transcript' called for URL: {youtube_url}")

    try:
        # --- Start of merged fetching logic ---
        video_id_match = re.search(r'(?:v=|/|embed/|shorts/)([0-9A-Za-z_-]{11})', youtube_url)
        if not video_id_match:
            logger.warning(f"Could not extract video ID from URL: {youtube_url}")
            response: Dict[str, Any] = {"status": "error", "message": "Could not extract video ID from URL."}
            return response

        video_id = video_id_match.group(1)

        # Try fetching the transcript
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            transcript_text = ' '.join([entry['text'] for entry in transcript_list])
            logger.info(f"Successfully fetched transcript for {youtube_url}")
            response: Dict[str, Any] = {"status": "success", "transcript": transcript_text}
            return response
        except Exception as fetch_error:
            # Log specific exceptions from the API call
            logger.warning(f"Could not fetch transcript for {youtube_url} (ID: {video_id}): {fetch_error}")
            response: Dict[str, Any] = {"status": "error", "message": f"Could not retrieve transcript: {fetch_error}"}
            return response
        # --- End of merged fetching logic ---

    except Exception as e:
        # Catch unexpected errors during the tool's execution (e.g., regex error?)
        logger.error(f"Unexpected error in get_youtube_transcript tool for {youtube_url}: {e}", exc_info=True)
        response: Dict[str, Any] = {"status": "error", "message": f"Failed to execute transcript tool due to an unexpected error: {str(e)}"}
        return response 

result = get_youtube_transcript("https://www.youtube.com/watch?v=KUEmEb71vzQ&t=5522s")
result


NameError: name 'logger' is not defined

# Gemini voice test

In [3]:
# -*- coding: utf-8 -*-
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
## Setup

To install the dependencies for this script, run:

``` 
pip install google-genai opencv-python pyaudio pillow mss
```

Before running this script, ensure the `GOOGLE_API_KEY` environment
variable is set to the api-key you obtained from Google AI Studio.

Important: **Use headphones**. This script uses the system default audio
input and output, which often won't include echo cancellation. So to prevent
the model from interrupting itself it is important that you use headphones. 

## Run

To run the script:

```
python Get_started_LiveAPI.py
```

The script takes a video-mode flag `--mode`, this can be "camera", "screen", or "none".
The default is "camera". To share your screen run:

```
python Get_started_LiveAPI.py --mode screen
```
"""

import asyncio
import base64
import io
import os
import sys
import traceback

import cv2
import pyaudio
import PIL.Image
import mss

import argparse

from google import genai

if sys.version_info < (3, 11, 0):
    import taskgroup, exceptiongroup

    asyncio.TaskGroup = taskgroup.TaskGroup
    asyncio.ExceptionGroup = exceptiongroup.ExceptionGroup

FORMAT = pyaudio.paInt16
CHANNELS = 1
SEND_SAMPLE_RATE = 16000
RECEIVE_SAMPLE_RATE = 24000
CHUNK_SIZE = 1024

MODEL = "models/gemini-2.0-flash-live-001"

DEFAULT_MODE = "camera"

client = genai.Client(http_options={"api_version": "v1beta"})

CONFIG = {"response_modalities": ["AUDIO"]}

pya = pyaudio.PyAudio()


class AudioLoop:
    def __init__(self, video_mode=DEFAULT_MODE, mic_index=None):
        self.video_mode = video_mode
        self.mic_index = mic_index

        self.audio_in_queue = None
        self.out_queue = None

        self.session = None

        self.send_text_task = None
        self.receive_audio_task = None
        self.play_audio_task = None

    async def send_text(self):
        while True:
            text = await asyncio.to_thread(
                input,
                "message > ",
            )
            if text.lower() == "q":
                break
            await self.session.send(input=text or ".", end_of_turn=True)

    def _get_frame(self, cap):
        # Read the frameq
        ret, frame = cap.read()
        # Check if the frame was read successfully
        if not ret:
            return None
        # Fix: Convert BGR to RGB color space
        # OpenCV captures in BGR but PIL expects RGB format
        # This prevents the blue tint in the video feed
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = PIL.Image.fromarray(frame_rgb)  # Now using RGB frame
        img.thumbnail([1024, 1024])

        image_io = io.BytesIO()
        img.save(image_io, format="jpeg")
        image_io.seek(0)

        mime_type = "image/jpeg"
        image_bytes = image_io.read()
        return {"mime_type": mime_type, "data": base64.b64encode(image_bytes).decode()}

    async def get_frames(self):
        # This takes about a second, and will block the whole program
        # causing the audio pipeline to overflow if you don't to_thread it.
        cap = await asyncio.to_thread(
            cv2.VideoCapture, 0
        )  # 0 represents the default camera

        while True:
            frame = await asyncio.to_thread(self._get_frame, cap)
            if frame is None:
                break

            await asyncio.sleep(1.0)

            await self.out_queue.put(frame)

        # Release the VideoCapture object
        cap.release()

    def _get_screen(self):
        sct = mss.mss()
        monitor = sct.monitors[0]

        i = sct.grab(monitor)

        mime_type = "image/jpeg"
        image_bytes = mss.tools.to_png(i.rgb, i.size)
        img = PIL.Image.open(io.BytesIO(image_bytes))

        image_io = io.BytesIO()
        img.save(image_io, format="jpeg")
        image_io.seek(0)

        image_bytes = image_io.read()
        return {"mime_type": mime_type, "data": base64.b64encode(image_bytes).decode()}

    async def get_screen(self):

        while True:
            frame = await asyncio.to_thread(self._get_screen)
            if frame is None:
                break

            await asyncio.sleep(1.0)

            await self.out_queue.put(frame)

    async def send_realtime(self):
        while True:
            msg = await self.out_queue.get()
            await self.session.send(input=msg)

    async def listen_audio(self):
        mic_info = (
            pya.get_device_info_by_index(self.mic_index)
            if self.mic_index is not None
            else pya.get_default_input_device_info()
        )
        self.audio_stream = await asyncio.to_thread(
            pya.open,
            format=FORMAT,
            channels=CHANNELS,
            rate=SEND_SAMPLE_RATE,
            input=True,
            input_device_index=mic_info["index"],
            frames_per_buffer=CHUNK_SIZE,
        )
        if __debug__:
            kwargs = {"exception_on_overflow": False}
        else:
            kwargs = {}
        while True:
            data = await asyncio.to_thread(self.audio_stream.read, CHUNK_SIZE, **kwargs)
            await self.out_queue.put({"data": data, "mime_type": "audio/pcm"})

    async def receive_audio(self):
        "Background task to reads from the websocket and write pcm chunks to the output queue"
        while True:
            turn = self.session.receive()
            async for response in turn:
                if data := response.data:
                    self.audio_in_queue.put_nowait(data)
                    continue
                if text := response.text:
                    print(text, end="")

            # If you interrupt the model, it sends a turn_complete.
            # For interruptions to work, we need to stop playback.
            # So empty out the audio queue because it may have loaded
            # much more audio than has played yet.
            while not self.audio_in_queue.empty():
                self.audio_in_queue.get_nowait()

    async def play_audio(self):
        stream = await asyncio.to_thread(
            pya.open,
            format=FORMAT,
            channels=CHANNELS,
            rate=RECEIVE_SAMPLE_RATE,
            output=True,
        )
        while True:
            bytestream = await self.audio_in_queue.get()
            await asyncio.to_thread(stream.write, bytestream)

    async def run(self):
        try:
            async with (
                client.aio.live.connect(model=MODEL, config=CONFIG) as session,
                asyncio.TaskGroup() as tg,
            ):
                self.session = session

                self.audio_in_queue = asyncio.Queue()
                self.out_queue = asyncio.Queue(maxsize=5)

                send_text_task = tg.create_task(self.send_text())
                tg.create_task(self.send_realtime())
                tg.create_task(self.listen_audio())
                if self.video_mode == "camera":
                    tg.create_task(self.get_frames())
                elif self.video_mode == "screen":
                    tg.create_task(self.get_screen())

                tg.create_task(self.receive_audio())
                tg.create_task(self.play_audio())

                await send_text_task
                raise asyncio.CancelledError("User requested exit")

        except asyncio.CancelledError:
            pass
        except ExceptionGroup as EG:
            self.audio_stream.close()
            traceback.print_exception(EG)


def list_input_devices():
    """Prints available input devices with their indices."""
    print("Available input devices:")
    for i in range(pya.get_device_count()):
        info = pya.get_device_info_by_index(i)
        if info["maxInputChannels"] > 0:
            print(f"{i}: {info['name']}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--mode",
        type=str,
        default=DEFAULT_MODE,
        help="pixels to stream from",
        choices=["camera", "screen", "none"],
    )
    parser.add_argument(
        "--mic",
        type=int,
        default=None,
        help="Index of the microphone input device to use. Run with --list-mics to see available devices.",
    )
    parser.add_argument(
        "--list-mics",
        action="store_true",
        help="List available microphone devices and exit.",
    )
    args = parser.parse_args()

    if args.list_mics:
        list_input_devices()
        sys.exit(0)

    main = AudioLoop(video_mode=args.mode, mic_index=args.mic)
    asyncio.run(main.run())
    


usage: ipykernel_launcher.py [-h] [--mode {camera,screen,none}] [--mic MIC]
                             [--list-mics]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/huyknguyen/Library/Jupyter/runtime/kernel-v3e3f2b32991aa2c0eeb04c5608a2facabaa4e1964.json


SystemExit: 2

/Users/huyknguyen/projects/huynguyen_web/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
#list microphone available:
for i in range(pya.get_device_count()):
    info = pya.get_device_info_by_index(i)
    if info["maxInputChannels"] > 0:
        print(f"{i}: {info['name']}")
        


1: Steve’s AirPods Pro #2
4: JLab JBuds Talk
6: USB Audio Device
7: BlackHole 2ch
8: MacBook Pro Microphone
10: Microsoft Teams Audio
